In [1]:
import os

os.makedirs("data/17flowers", exist_ok=True)
os.makedirs("data/102flowers", exist_ok=True)

In [ ]:
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/17flowers.tgz
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/datasplits.mat
!wget -P data/17flowers https://thor.robots.ox.ac.uk/datasets/flowers-17/trimaps.tgz

In [ ]:
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/102flowers.tgz
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/imagelabels.mat
!wget -P data/102flowers https://thor.robots.ox.ac.uk/datasets/flowers-102/setid.mat

In [ ]:
!tar -xzf data/17flowers/17flowers.tgz -C data/17flowers

In [ ]:
!tar -xzf data/102flowers/102flowers.tgz -C data/102flowers

In [ ]:
!rm data/102flowers/102flowers.tgz
!rm data/17flowers/17flowers.tgz

## Color constancy dataset using fc4

In [ ]:
import os

os.makedirs("data/cc/17flowers", exist_ok=True)
os.makedirs("data/cc/102flowers", exist_ok=True)

In [4]:
!python3 ./thirdparty/fc4-python3/fc4.py test ./pretrained_colorchecker/colorchecker_fold1and2.ckpt -1 ./data/17flowers ./data/cc/17flowers

Traceback (most recent call last):
  File "/media/shiohiyoko/sandisk/IMLEX/DL/DL_CC_Classification/./thirdparty/fc4-python3/fc4.py", line 8, in <module>
    import tensorflow as tf
ModuleNotFoundError: No module named 'tensorflow'
